### 네이버 현재 상영영화 크롤링

In [ ]:
//*[@id="content"]/div[1]/div[1]/div[3]/ul/li[1]/dl/dt/a

In [7]:
import scrapy
from scrapy.http import TextResponse

In [1]:
# 1. selector 찾기 : link, title, count, star

In [1]:
# link
url = "https://movie.naver.com/movie/running/current.nhn"
req = requests.get(url)
response = TextResponse(req.url, body=req.text, encoding="utf-8")
links = response.xpath(
    '//*[@id="content"]/div[1]/div[1]/div[3]/ul/li/dl/dt/a/@href').extract()
links = [response.urljoin(link) for link in links]
links[-3:]

NameError: name 'TextResponse' is not defined

In [19]:
link = links[0]
link

'https://movie.naver.com/movie/bi/mi/basic.nhn?code=189537'

In [20]:
# content : title, count, star
req = requests.get(link)
response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=189537>

In [39]:
title = response.xpath(
    '//*[@id="content"]/div[1]/div[2]/div[1]/h3/a[1]/text()')[0].extract()
count = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[5]/div/p[2]/text()')[0].extract()
star = response.xpath(
    '//*[@id="actualPointPersentBasic"]/div/em/text()').extract()
star = "".join(star)
title, count, star

('#살아있다', '1,291,102명', '7.97')

In [2]:
# 2. 프로젝트 생성

In [28]:
!scrapy startproject naver_movie

New Scrapy project 'naver_movie', using template directory '/usr/local/anaconda3/lib/python3.7/site-packages/scrapy/templates/project', created in:
    /Users/rada/Documents/lecture/dss/dss_13/code/dss_13/04_scrapy/naver_movie

You can start your first spider with:
    cd naver_movie
    scrapy genspider example example.com


In [3]:
# 3. items.py 코드 작성

In [31]:
%%writefile naver_movie/naver_movie/items.py
import scrapy

class NaverMovieItem(scrapy.Item):
    title = scrapy.Field()
    count = scrapy.Field()
    star = scrapy.Field()
    link = scrapy.Field()

Overwriting naver_movie/naver_movie/items.py


In [4]:
# 4. spider.py 코드 작성

In [37]:
%%writefile naver_movie/naver_movie/spiders/spider.py
import scrapy
from naver_movie.items import NaverMovieItem

class Spider(scrapy.Spider):
    name = "NaverMovie"
    allow_domain = ["https://movie.naver.com"]
    start_urls = ["https://movie.naver.com/movie/running/current.nhn"]
    
    def parse(self, response):
        links = response.xpath(
            '//*[@id="content"]/div[1]/div[1]/div[3]/ul/li/dl/dt/a/@href').extract()    
        links = [response.urljoin(link) for link in links]
        for link in links:
            yield scrapy.Request(link, callback=self.parse_content)
    
    def parse_content(self, response):
        item = NaverMovieItem()
        item["title"] = response.xpath(
            '//*[@id="content"]/div[1]/div[2]/div[1]/h3/a[1]/text()')[0].extract()
        try:
            item["count"] = response.xpath(
                '//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[5]/div/p[2]/text()')[0].extract()
        except:
            item["count"] = "0명"
        star = response.xpath(
            '//*[@id="actualPointPersentBasic"]/div/em/text()').extract()
        item["star"] = "".join(star)
        item["link"] = response.url
        yield item

Overwriting naver_movie/naver_movie/spiders/spider.py


In [5]:
# 5. settings.py 설정 변경 : awk

In [35]:
!sed -i .bak 's/ROBOTSTXT_OBEY = True/ROBOTSTXT_OBEY = False/' naver_movie/naver_movie/settings.py

In [6]:
# 6. scrapy 실행

In [33]:
%%writefile run.sh
cd naver_movie
scrapy crawl NaverMovie

Overwriting run.sh


In [38]:
!source run.sh

2020-07-03 17:00:49 [scrapy.utils.log] INFO: Scrapy 2.2.0 started (bot: naver_movie)
2020-07-03 17:00:49 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.6 (default, Jan  8 2020, 13:42:34) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1  11 Sep 2018), cryptography 2.8, Platform Darwin-19.4.0-x86_64-i386-64bit
2020-07-03 17:00:49 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-07-03 17:00:49 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'naver_movie',
 'NEWSPIDER_MODULE': 'naver_movie.spiders',
 'SPIDER_MODULES': ['naver_movie.spiders']}
2020-07-03 17:00:49 [scrapy.extensions.telnet] INFO: Telnet Password: b2ad22670033caf1
2020-07-03 17:00:49 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.

2020-07-03 17:00:51 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=11093>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=11093',
 'star': '',
 'title': '플라이'}
2020-07-03 17:00:51 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://movie.naver.com/movie/bi/mi/basic.nhn?code=23847> (referer: https://movie.naver.com/movie/running/current.nhn)
2020-07-03 17:00:51 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=181099>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=181099',
 'star': '',
 'title': '바다로 가자'}
2020-07-03 17:00:51 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=194777>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=194777',
 'star': '',
 'title': '포스 오브 네이쳐'}
2020-07-03 17:00:51 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.

2020-07-03 17:00:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://movie.naver.com/movie/bi/mi/basic.nhn?code=172176> (referer: https://movie.naver.com/movie/running/current.nhn)
2020-07-03 17:00:52 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=188765>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=188765',
 'star': '8.70',
 'title': '문신을 한 신부님'}
2020-07-03 17:00:52 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=66440>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=66440',
 'star': '9.13',
 'title': '슬럼독 밀리어네어'}
2020-07-03 17:00:52 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=155123>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=155123',
 'star': '9.36',
 'title': '미스 사이공: 25주년 특별 공연'}
2020-07-03 17:00:52 [scrapy.core.engine] DEBUG: Cr

2020-07-03 17:00:53 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=183427>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=183427',
 'star': '7.75',
 'title': '조금씩, 천천히 안녕'}
2020-07-03 17:00:53 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=107352>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=107352',
 'star': '7.99',
 'title': '자객 섭은낭'}
2020-07-03 17:00:53 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://movie.naver.com/movie/bi/mi/basic.nhn?code=156322> (referer: https://movie.naver.com/movie/running/current.nhn)
2020-07-03 17:00:53 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=189001>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=189001',
 'star': '8.91',
 'title': '인비저블맨'}
2020-07-03 17:00:53 [scrapy.core.scraper] DEBUG: Scraped from 

2020-07-03 17:00:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://movie.naver.com/movie/bi/mi/basic.nhn?code=147040> (referer: https://movie.naver.com/movie/running/current.nhn)
2020-07-03 17:00:54 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=189463>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=189463',
 'star': '9.24',
 'title': '몽마르트 파파'}
2020-07-03 17:00:54 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=189438>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=189438',
 'star': '9.40',
 'title': '바람의 언덕'}
2020-07-03 17:00:54 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=189139>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=189139',
 'star': '8.88',
 'title': '리암 갤러거'}
2020-07-03 17:00:54 [scrapy.core.scraper] DEBUG: Scraped from <20

2020-07-03 17:00:55 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=184859>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=184859',
 'star': '10.0',
 'title': '물의 기억'}
2020-07-03 17:00:55 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=95203>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=95203',
 'star': '8.63',
 'title': '로렌스 애니웨이 '}
2020-07-03 17:00:55 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=78872>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=78872',
 'star': '8.00',
 'title': '돼지의 왕'}
2020-07-03 17:00:55 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=187979>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=187979',
 'star': '8.86',
 'title': '나의 노래는 멀리멀리'}
20

2020-07-03 17:00:56 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=193076>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=193076',
 'star': '',
 'title': '벨빌 캅'}
2020-07-03 17:00:56 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=146548>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=146548',
 'star': '8.66',
 'title': '태풍이 지나가고'}
2020-07-03 17:00:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://movie.naver.com/movie/bi/mi/basic.nhn?code=90537> (referer: https://movie.naver.com/movie/running/current.nhn)
2020-07-03 17:00:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://movie.naver.com/movie/bi/mi/basic.nhn?code=84375> (referer: https://movie.naver.com/movie/running/current.nhn)
2020-07-03 17:00:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://movie.naver.com/movie/bi/mi/basic.nhn?code=10243> (referer:

2020-07-03 17:00:57 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=181114>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=181114',
 'star': '8.36',
 'title': '날씨의 아이'}
2020-07-03 17:00:57 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=63513>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=63513',
 'star': '9.43',
 'title': '시간을 달리는 소녀'}
2020-07-03 17:00:57 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=194004>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=194004',
 'star': '9.00',
 'title': '16세의 사운드트랙'}
2020-07-03 17:00:57 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=113170>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=113170',
 'star': '8.84',
 'title': '마담 프루스트

2020-07-03 17:00:58 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=182099>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=182099',
 'star': '10.0',
 'title': '에이브의 쿠킹 다이어리'}
2020-07-03 17:00:58 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=68498>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=68498',
 'star': '8.92',
 'title': '피아노의 숲'}
2020-07-03 17:00:58 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://movie.naver.com/movie/bi/mi/basic.nhn?code=189712> (referer: https://movie.naver.com/movie/running/current.nhn)
2020-07-03 17:00:58 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://movie.naver.com/movie/bi/mi/basic.nhn?code=34093> (referer: https://movie.naver.com/movie/running/current.nhn)
2020-07-03 17:00:58 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://movie.naver.com/movie/bi/mi/basic.nhn?code=84917> 

2020-07-03 17:00:59 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=168031>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=168031',
 'star': '10.0',
 'title': '해피 디 데이'}
2020-07-03 17:00:59 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=37742>
{'count': '0명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=37742',
 'star': '',
 'title': '트로이'}
2020-07-03 17:00:59 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=194807>
{'count': '1,816명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=194807',
 'star': '',
 'title': '아무튼, 아담'}
2020-07-03 17:00:59 [scrapy.core.scraper] DEBUG: Scraped from <200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=158885>
{'count': '225,478명',
 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=158885',
 'star': '9.13',
 'title': '콜 미 바이 유어 네임'}